In [1]:
import pandas as pd
from tqdm import tqdm
import os
import ast
import re
os.environ['http_proxy'] = "http://xen03.iitd.ac.in:3128"
os.environ['https_proxy'] = "http://xen03.iitd.ac.in:3128"

from utils import postprocess_llm_pred, dashed_line

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/amey/miniconda3/envs/cs-eval/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-13 10:18:38.840784: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 10:18:38.897226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 10:18:38.897269: E external/local_xla/xla/stream

In [ ]:
df_human_annotation2 = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/Human Evaluation - EMNLP 2024 - annotations (1).csv')
df_human_annotation2.shape
# df_human_annotation2['Rank'].isna().sum()
df_human_annotation2 = df_human_annotation2[df_human_annotation2['Rank'].notna()]
df_human_annotation2 = df_human_annotation2.rename(columns={'Rank': 'rank_human_annotation'})
df_human_annotation2.head(6)
assert df_human_annotation2.shape[0] == 310

In [2]:
def get_val(d):
    """
    Returns the value from a dictionary containing a single key-value pair.
    
    Args:
    d (dict): A dictionary with a single key-value pair.
    
    Returns:
    The value of the single key-value pair in the dictionary.
    """
    assert isinstance(d, str)
    d = ast.literal_eval(d)
    return float(next(iter(d.values())))


def extract_score(input_string: str, key='Relevance') -> float:
    if not isinstance(input_string, str):
        print(
            f"Invalid input: {input_string}"
        )
        return -1
    """
    try:
        input = ast.literal_eval(input)
        score = input[key]
        score = score.strip().replace(' ','')
        score = float(score)
        return score
    except:
        return -1
    """
     # Use regex to find the number in the string
    match = re.search(r'\d+', input_string)
    
    # If a number is found, convert it to float and return it
    if match:
        return float(match.group())
    else:
        return -1

#### GPT-4-o

In [3]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/gpt-4o.pkl')
df_pred.head(1)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,GPTScore_relevance_score,GPTScore_coherence_score,GPTScore_aggressiveness_score,GPTScore_suitableness_score,"bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)",prediction_gpt-4o_relevance_score,prediction_gpt-4o_coherence_score,prediction_gpt-4o_suitableness_score,prediction_gpt-4o_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,4.0,1.0,2.0,2.0,0.002085,0.001052,"{""Relevance"": ""4""}","{""Argument Coherence"": ""2""}","{""Choose or Not"": ""2""}","{\n""Aggressiveness"": ""1""\n}"


In [116]:
df_pred['prediction_gpt-4o_coherence_score'].value_counts()

prediction_gpt-4o_coherence_score
{"Argument Coherence": "2"}            283
{"Argument Coherence": "3"}            243
{"Argument Coherence": "4"}            222
{"Argument Coherence": "1"}             54
{\n  "Argument Coherence": "2"\n}       42
{\n  "Argument Coherence": "3"\n}       37
{\n  "Argument Coherence": "4"\n}       32
{\n  "Argument Coherence": "1"\n}       20
{\n    "Argument Coherence": "4"\n}     14
{\n    "Argument Coherence": "2"\n}     13
{\n"Argument Coherence": "3"\n}         10
{\n    "Argument Coherence": "3"\n}      9
{\n"Argument Coherence": "2"\n}          8
{\n    "Argument Coherence": "1"\n}      4
{\n"Argument Coherence": "1"\n}          3
{\n"Argument Coherence": "4"\n}          3
Name: count, dtype: int64

In [117]:
df_pred['prediction_gpt-4o_suitableness_score'] = df_pred['prediction_gpt-4o_suitableness_score'].apply(lambda x: extract_score(x, key="Choose or Not"))
df_pred['prediction_gpt-4o_relevance_score'] = df_pred['prediction_gpt-4o_relevance_score'].apply(lambda x: extract_score(x, key="Relevance"))
df_pred['prediction_gpt-4o_coherence_score'] = df_pred['prediction_gpt-4o_coherence_score'].apply(lambda x: extract_score(x, key="Argument Coherence"))
df_pred['prediction_gpt-4o_aggressiveness_score'] = df_pred['prediction_gpt-4o_aggressiveness_score'].apply(lambda x: extract_score(x, key="Aggressiveness"))

Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None
Invalid input: None


In [118]:
source_columns = ['gpt-4_suitableness_score', 'gpt-4_relevance_score', 'gpt-4_coherence_score', 'gpt-4_aggressiveness_score']
pred_columns = ['prediction_gpt-4o_suitableness_score', 'prediction_gpt-4o_relevance_score', 'prediction_gpt-4o_coherence_score','prediction_gpt-4o_aggressiveness_score']

# for col in pred_columns:
#     print(df_pred[col].value_counts().items)

for source_col in source_columns:
    print(df_pred[source_col].value_counts().items)

print(dashed_line)
print(dashed_line)

for source_col, pred_col in zip(source_columns, pred_columns):
    df_pred[source_col] = df_pred.apply(lambda row: row[pred_col] if (pd.isna(row[source_col])) else row[source_col], axis=1)
    print(df_pred[source_col].value_counts().items)

<bound method Series.items of gpt-4_suitableness_score
2.0    2729
1.0    1407
3.0    1393
Name: count, dtype: int64>
<bound method Series.items of gpt-4_relevance_score
5.0    3701
4.0     898
1.0     416
3.0     331
2.0     183
Name: count, dtype: int64>
<bound method Series.items of gpt-4_coherence_score
1.0    2141
2.0    1350
3.0    1205
4.0     658
5.0     175
Name: count, dtype: int64>
<bound method Series.items of gpt-4_aggressiveness_score
1.0    3281
5.0     668
2.0     584
3.0     542
4.0     454
Name: count, dtype: int64>
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
<bound method Series.items of gpt-4_suitableness_score
 2.0    3348
 3.0    1656
 1.0    1522
-1.0    1400
Name: count, dtype: int64>
<bound method Series.items of gpt-4_relevance_score
 5.0    4360
-1.0    1400
 4.0    1102
 1.0     451
 3.0     400
 2.0   

In [121]:
df_pred[df_pred['gpt-4_relevance_score'] == -1]['annotator'].value_counts()

annotator
(shaily,rishabh)    1000
(aswini)             400
Name: count, dtype: int64

In [123]:
df_pred.to_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv', index=False)

#### GPT4-o-zeroshot

In [7]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/gpt-4o_zeroshot.pkl')
print(df_pred.shape)
df_pred.head(1)

df_pred['prediction_gpt-4o_relevance_score'].isna().sum()

# pred_columns = [
#     'prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score', 
#     'prediction_gpt-4o_relevance_score', 
#     'prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
#     'prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score'
# ]

# for col in pred_columns:
#     df_pred[f"zeroshot_{col}"] = df_pred[col]

# dataset = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv')
# print(dataset.shape)
# dataset.head(1)

# cols = [col for col in df_pred.columns if col not in dataset.columns] + ['uuid']
# df_pred = pd.merge(dataset, df_pred[cols], on=['uuid'])

# assert df_pred.shape[0] == dataset.shape[0]

(7926, 79)


6425

#### Mistral

In [1]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/mistralai-Mistral-7B-Instruct-v0.3.pkl')
print(df_pred.shape)
df_pred.head(1)
# df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score'].isna().sum()

NameError: name 'pd' is not defined

In [16]:
df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'].iloc[0]

ChatCompletionResponse(id='88e2737a6be6338a-DEL', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1717446732, model='mistralai/Mistral-7B-Instruct-v0.3', choices=[ChatCompletionChoicesData(index=0, logprobs=LogprobsPart(tokens=[' {"', 'Argument', ' Co', 'her', 'ence', '":', ' ', '3', '}', '\n', '\n', 'This', ' coun', 'ters', 'pe', 'ech', ' demonstr', 'ates', ' a', ' moder', 'ately', ' co', 'herent', ' approach', '.', ' It', ' presents', ' a', ' clear', ' and', ' direct', ' question'], token_logprobs=[-0.0003437996, -0.00016081333, -1.7881393e-06, -2.6226044e-06, -3.5762787e-07, -1.0728836e-06, -0.055725098, -0.41259766, -0.0035133362, -0.02116394, -0.047698975, -2.1835938, -0.019134521, -1.4305115e-06, -0.00024282932, -0.00018048286, -2.8378906, -3.5762787e-07, -0.02079773, -1.140625, -7.867813e-06, -0.00095129013, -0.00017309189, -5.1601562, -2.3808594, -0.6069336, -0.92333984, -0.097595215, -1.2333984, -2.375, -0.47558594, -0.203125], token_ids=[10598, 9717, 3860, 1431

In [17]:
def extract_response(response):
    try:
        return response.choices[0].message.content
    except:
        return -1
    # for x in response.choices:
    #     print(x.message.content)
    #     print(x)

sample = df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'].iloc[0]
extract_response(sample)

' {"Argument Coherence": 3}\n\nThis counterspeech demonstrates a moderately coherent approach. It presents a clear and direct question'

In [18]:
df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'] = df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'].apply(
    lambda x: extract_response(x)
)

df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'] = df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'].apply(
    lambda x: extract_score(x)
)

Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid in

In [19]:
df_pred['prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score'].value_counts()

prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score
-1.0    6026
 4.0     819
 5.0     578
 3.0     316
 2.0     142
 1.0      45
Name: count, dtype: int64

prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score
-1.0    6026
 4.0     819
 5.0     578
 3.0     316
 2.0     142
 1.0      45
Name: count, dtype: int64

#### Meta-LLama-COT

In [78]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/mistralai-Mistral-7B-Instruct-v0.3.pkl')
print(df_pred.shape)
df_pred.head(1)


(7926, 9)


,index,hatespeech,counterspeech,predicted_counterspeech,uuid,prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score
0,0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,91761985-3ea5-482d-b20f-d855df5909d0,id='88e2511e5b6e33ae-DEL' object=<ObjectType.C...,id='88e2512eea0333ae-DEL' object=<ObjectType.C...,id='88e251395bfd33ae-DEL' object=<ObjectType.C...,id='88e2512b4e5e33ae-DEL' object=<ObjectType.C...


In [79]:
dataset = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv')
print(dataset.shape)
dataset.head(1)

(7926, 79)


,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,GPTScore_relevance_score,GPTScore_coherence_score,GPTScore_aggressiveness_score,GPTScore_suitableness_score,"bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)",prediction_gpt-4o_relevance_score,prediction_gpt-4o_coherence_score,prediction_gpt-4o_suitableness_score,prediction_gpt-4o_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,4.0,1.0,2.0,2.0,0.002085,0.001052,-1.0,-1.0,-1.0,-1.0


In [80]:
cols = [col for col in df_pred.columns if col not in dataset.columns] + ['uuid']
df_pred = pd.merge(dataset, df_pred[cols], on=['uuid'])

assert df_pred.shape[0] == dataset.shape[0]

In [81]:
def extract_response(response):
    try:
        return response.choices[0].message.content
    except:
        return -1
    # for x in response.choices:
    #     print(x.message.content)
    #     print(x)

sample = df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'].iloc[0]
extract_response(sample)

'{"Relevance": 4}'

In [82]:
# df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'] = df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'].apply(
#     lambda x: extract_response(x)
# )

# df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'] = df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'].apply(
#     lambda x: extract_score(x)
# )

# df_pred['prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'].value_counts()

In [83]:
# df_pred['prediction_gpt-4o_suitableness_score'] = df_pred['prediction_gpt-4o_suitableness_score'].apply(lambda x: extract_score(x, key="Choose or Not"))
# df_pred['prediction_gpt-4o_relevance_score'] = df_pred['prediction_gpt-4o_relevance_score'].apply(lambda x: extract_score(x, key="Relevance"))
# df_pred['prediction_gpt-4o_coherence_score'] = df_pred['prediction_gpt-4o_coherence_score'].apply(lambda x: extract_score(x, key="Argument Coherence"))
# df_pred['prediction_gpt-4o_aggressiveness_score'] = df_pred['prediction_gpt-4o_aggressiveness_score'].apply(lambda x: extract_score(x, key="Aggressiveness"))

In [84]:
source_columns = ['gpt-4_suitableness_score', 'gpt-4_relevance_score', 'gpt-4_coherence_score', 'gpt-4_aggressiveness_score']
pred_columns = [
    'prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score', 
    'prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score', 
    'prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
    'prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score'
]

# for col in pred_columns:
#     print(df_pred[col].value_counts().items)

for source_col in source_columns:
    print(df_pred[source_col].value_counts().items)

print(dashed_line)
print(dashed_line)

for source_col, pred_col in zip(source_columns, pred_columns):
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_response(x))
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_score(x))
    print(df_pred[pred_col].value_counts().items)

<bound method Series.items of gpt-4_suitableness_score
 2.0    3348
 3.0    1656
 1.0    1522
-1.0    1400
Name: count, dtype: int64>
<bound method Series.items of gpt-4_relevance_score
 5.0    4360
-1.0    1400
 4.0    1102
 1.0     451
 3.0     400
 2.0     213
Name: count, dtype: int64>
<bound method Series.items of gpt-4_coherence_score
 1.0    2222
 2.0    1696
 3.0    1504
-1.0    1400
 4.0     929
 5.0     175
Name: count, dtype: int64>
<bound method Series.items of gpt-4_aggressiveness_score
 1.0    4218
-1.0    1400
 5.0     671
 2.0     627
 3.0     553
 4.0     457
Name: count, dtype: int64>
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid input: -1
Invalid 

In [85]:
df_pred.head(3)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,"bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)",prediction_gpt-4o_relevance_score,prediction_gpt-4o_coherence_score,prediction_gpt-4o_suitableness_score,prediction_gpt-4o_aggressiveness_score,prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,0.002085,0.001052,-1.0,-1.0,-1.0,-1.0,4.0,4.0,2.0,1.0
1,1,1.0,Islam is a destructive and violent religion.,It is important to approach religious discussi...,I understand that you may have concerns about ...,Positive,2.333333,3.666667,4.333333,1.000000,...,NaN,NaN,5.0,3.0,2.0,1.0,5.0,4.0,3.0,2.0
2,2,2.0,"Muslims are all terrorists, they have nothing ...",There is no evidence to suggest that terrorist...,It is important to recognize that terrorism is...,Denouncing,2.666667,3.333333,3.333333,1.000000,...,0.001275,0.000641,-1.0,-1.0,-1.0,-1.0,5.0,4.0,2.0,1.0


In [86]:
df_pred.to_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv', index=False)

#### Meta-LLama-zeroshot

In [4]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/meta-llama-Llama-3-8b-chat-hf_zeroshot.pkl')
print(df_pred.shape)
df_pred.head(1)

pred_columns = [
    'prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score', 
    'prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score', 
    'prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
    'prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score'
]

for col in pred_columns:
    df_pred[f"zeroshot_{col}"] = df_pred[col]

dataset = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv')
print(dataset.shape)
dataset.head(1)

cols = [col for col in df_pred.columns if col not in dataset.columns] + ['uuid']
df_pred = pd.merge(dataset, df_pred[cols], on=['uuid'])

assert df_pred.shape[0] == dataset.shape[0]

(7926, 9)
(7926, 83)


In [5]:
df_pred.head(1)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,prediction_gpt-4o_suitableness_score,prediction_gpt-4o_aggressiveness_score,prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,-1.0,-1.0,4.0,4.0,2.0,1.0,id='88e2fa8fcaae33b4-DEL' object=<ObjectType.C...,id='88e2fa76bab233b4-DEL' object=<ObjectType.C...,id='88e2fa8b1f9333b4-DEL' object=<ObjectType.C...,id='88e2fa7effa433b4-DEL' object=<ObjectType.C...


In [6]:
def extract_response(response):
    try:
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return -1
    # for x in response.choices:
    #     print(x.message.content)
    #     print(x)

sample = df_pred['zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score'].iloc[10]
extract_response(sample)

'{"Relevance": 4}'

In [7]:
source_columns = ['gpt-4_suitableness_score', 'gpt-4_relevance_score', 'gpt-4_coherence_score', 'gpt-4_aggressiveness_score']
pred_columns = [
    'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score', 
    'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score', 
    'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
    'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score'
]

# for col in pred_columns:
#     print(df_pred[col].value_counts().items)

for source_col in source_columns:
    print(df_pred[source_col].value_counts().items)

print(dashed_line)
print(dashed_line)

for source_col, pred_col in zip(source_columns, pred_columns):
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_response(x))
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_score(x))
    print(df_pred[pred_col].value_counts().items)

<bound method Series.items of gpt-4_suitableness_score
 2.0    3348
 3.0    1656
 1.0    1522
-1.0    1400
Name: count, dtype: int64>
<bound method Series.items of gpt-4_relevance_score
 5.0    4360
-1.0    1400
 4.0    1102
 1.0     451
 3.0     400
 2.0     213
Name: count, dtype: int64>
<bound method Series.items of gpt-4_coherence_score
 1.0    2222
 2.0    1696
 3.0    1504
-1.0    1400
 4.0     929
 5.0     175
Name: count, dtype: int64>
<bound method Series.items of gpt-4_aggressiveness_score
 1.0    4218
-1.0    1400
 5.0     671
 2.0     627
 3.0     553
 4.0     457
Name: count, dtype: int64>
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneTyp

In [8]:
df_pred.head(3)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,prediction_gpt-4o_suitableness_score,prediction_gpt-4o_aggressiveness_score,prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,-1.0,-1.0,4.0,4.0,2.0,1.0,3.0,5.0,4.0,1.0
1,1,1.0,Islam is a destructive and violent religion.,It is important to approach religious discussi...,I understand that you may have concerns about ...,Positive,2.333333,3.666667,4.333333,1.000000,...,2.0,1.0,5.0,4.0,3.0,2.0,2.0,5.0,4.0,1.0
2,2,2.0,"Muslims are all terrorists, they have nothing ...",There is no evidence to suggest that terrorist...,It is important to recognize that terrorism is...,Denouncing,2.666667,3.333333,3.333333,1.000000,...,-1.0,-1.0,5.0,4.0,2.0,1.0,2.0,5.0,4.0,1.0


In [9]:
df_pred.to_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv', index=False)

#### Mistral zeroshot

In [17]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/mistralai-Mistral-7B-Instruct-v0.3_zeroshot.pkl')
print(df_pred.shape)
df_pred.head(1)

pred_columns = [
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score', 
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score', 
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score',
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score'
]

for col in pred_columns:
    df_pred = df_pred.rename(columns={
        col: f"zeroshot_{col}"
    })
    # df_pred[] = df_pred[col]

dataset = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv')
print(dataset.shape)
dataset.head(1)

cols = [col for col in df_pred.columns if col not in dataset.columns] + ['uuid']
df_pred = pd.merge(dataset, df_pred[cols], on=['uuid'])

assert df_pred.shape[0] == dataset.shape[0]

df_pred.head(1)

(7926, 9)
(7926, 87)


,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,2.0,1.0,3.0,5.0,4.0,1.0,id='88e2ff91bddf33a8-DEL' object=<ObjectType.C...,id='88e2ffa1785a33a8-DEL' object=<ObjectType.C...,id='88e2ffabefc433a8-DEL' object=<ObjectType.C...,id='88e2ff950fc033a8-DEL' object=<ObjectType.C...


In [18]:
def extract_response(response):
    try:
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return -1
    # for x in response.choices:
    #     print(x.message.content)
    #     print(x)

sample = df_pred['zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score'].iloc[10]
extract_response(sample)

' {"Choose or Not": 3}'

In [19]:
source_columns = ['gpt-4_suitableness_score', 'gpt-4_relevance_score', 'gpt-4_coherence_score', 'gpt-4_aggressiveness_score']
pred_columns = [
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score', 
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score', 
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score',
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score'
]

# for col in pred_columns:
#     print(df_pred[col].value_counts().items)

for source_col in source_columns:
    print(df_pred[source_col].value_counts().items)

print(dashed_line)
print(dashed_line)

for source_col, pred_col in zip(source_columns, pred_columns):
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_response(x))
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_score(x))
    print(df_pred[pred_col].value_counts().items)

<bound method Series.items of gpt-4_suitableness_score
 2.0    3348
 3.0    1656
 1.0    1522
-1.0    1400
Name: count, dtype: int64>
<bound method Series.items of gpt-4_relevance_score
 5.0    4360
-1.0    1400
 4.0    1102
 1.0     451
 3.0     400
 2.0     213
Name: count, dtype: int64>
<bound method Series.items of gpt-4_coherence_score
 1.0    2222
 2.0    1696
 3.0    1504
-1.0    1400
 4.0     929
 5.0     175
Name: count, dtype: int64>
<bound method Series.items of gpt-4_aggressiveness_score
 1.0    4218
-1.0    1400
 5.0     671
 2.0     627
 3.0     553
 4.0     457
Name: count, dtype: int64>
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneType' object has no attribute 'choices'
'NoneTyp

In [20]:
df_pred.head(3)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score,zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score,zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,2.0,1.0,3.0,5.0,4.0,1.0,5.0,3.0,3.0,2.0
1,1,1.0,Islam is a destructive and violent religion.,It is important to approach religious discussi...,I understand that you may have concerns about ...,Positive,2.333333,3.666667,4.333333,1.000000,...,3.0,2.0,2.0,5.0,4.0,1.0,5.0,4.0,3.0,2.0
2,2,2.0,"Muslims are all terrorists, they have nothing ...",There is no evidence to suggest that terrorist...,It is important to recognize that terrorism is...,Denouncing,2.666667,3.333333,3.333333,1.000000,...,2.0,1.0,2.0,5.0,4.0,1.0,5.0,5.0,3.0,2.0


In [21]:
df_pred.to_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv', index=False)

#### Normal Mistral

In [ ]:
df_pred = pd.read_pickle('/home/amey/depository/cs-eval/predictions/mistralai-Mistral-7B-Instruct-v0.3_zeroshot.pkl')
print(df_pred.shape)
df_pred.head(1)

pred_columns = [
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score', 
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score', 
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score',
    'prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score'
]

for col in pred_columns:
    df_pred = df_pred.rename(columns={
        col: f"zeroshot_{col}"
    })
    # df_pred[] = df_pred[col]

dataset = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated.csv')
print(dataset.shape)
dataset.head(1)

cols = [col for col in df_pred.columns if col not in dataset.columns] + ['uuid']
df_pred = pd.merge(dataset, df_pred[cols], on=['uuid'])

assert df_pred.shape[0] == dataset.shape[0]

df_pred.head(1)

In [ ]:
def extract_response(response):
    try:
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return -1
    # for x in response.choices:
    #     print(x.message.content)
    #     print(x)

sample = df_pred['zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score'].iloc[10]
extract_response(sample)

In [ ]:
source_columns = ['gpt-4_suitableness_score', 'gpt-4_relevance_score', 'gpt-4_coherence_score', 'gpt-4_aggressiveness_score']
pred_columns = [
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score', 
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score', 
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score',
    'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score'
]

# for col in pred_columns:
#     print(df_pred[col].value_counts().items)

for source_col in source_columns:
    print(df_pred[source_col].value_counts().items)

print(dashed_line)
print(dashed_line)

for source_col, pred_col in zip(source_columns, pred_columns):
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_response(x))
    df_pred[pred_col] = df_pred[pred_col].apply(lambda x: extract_score(x))
    print(df_pred[pred_col].value_counts().items)